# Transfer Learning and Fine-Tuning

## Introduction

**Transfer learning** is a machine learning paradigm where a model trained on one task (the *source task*) is repurposed as the starting point for a model on a different but related task (the *target task*). This approach leverages the knowledge encoded in pre-trained models to accelerate learning and improve performance, particularly when labeled data for the target task is scarce.

## Mathematical Foundation

### Domain and Task Definitions

Let us formalize the transfer learning framework. A **domain** $\mathcal{D}$ consists of a feature space $\mathcal{X}$ and a marginal probability distribution $P(X)$, where $X = \{x_1, x_2, \ldots, x_n\} \in \mathcal{X}$:

$$\mathcal{D} = \{\mathcal{X}, P(X)\}$$

A **task** $\mathcal{T}$ is defined by a label space $\mathcal{Y}$ and a predictive function $f(\cdot)$:

$$\mathcal{T} = \{\mathcal{Y}, f(\cdot)\}$$

where $f(x) = P(Y|X)$ can be learned from training data.

### Transfer Learning Objective

Given a source domain $\mathcal{D}_S$ with task $\mathcal{T}_S$ and a target domain $\mathcal{D}_T$ with task $\mathcal{T}_T$, transfer learning aims to improve the learning of the target predictive function $f_T(\cdot)$ using knowledge from $\mathcal{D}_S$ and $\mathcal{T}_S$, where $\mathcal{D}_S \neq \mathcal{D}_T$ or $\mathcal{T}_S \neq \mathcal{T}_T$.

### Neural Network Representation

In deep learning, a neural network can be decomposed into a **feature extractor** $\phi(\cdot; \theta_\phi)$ and a **classifier** $g(\cdot; \theta_g)$:

$$f(x) = g(\phi(x; \theta_\phi); \theta_g)$$

The feature extractor maps inputs to a learned representation space:

$$\phi: \mathcal{X} \rightarrow \mathbb{R}^d$$

where $d$ is the dimensionality of the feature representation.

## Fine-Tuning Strategies

### Feature Extraction (Frozen Backbone)

In this approach, the pre-trained feature extractor parameters $\theta_\phi$ are frozen, and only the classifier parameters $\theta_g$ are updated:

$$\theta_g^* = \arg\min_{\theta_g} \mathcal{L}(g(\phi(X; \theta_\phi^{\text{pre}}); \theta_g), Y)$$

### Full Fine-Tuning

All parameters are updated with a smaller learning rate $\eta_{\text{fine}}$:

$$\theta^* = \arg\min_{\theta_\phi, \theta_g} \mathcal{L}(f(X; \theta_\phi, \theta_g), Y)$$

### Layer-wise Learning Rate Decay

Earlier layers (closer to input) receive smaller learning rates. For layer $l$:

$$\eta_l = \eta_{\text{base}} \cdot \gamma^{L-l}$$

where $L$ is the total number of layers and $\gamma < 1$ is the decay factor.

### Gradual Unfreezing

Layers are progressively unfrozen during training, starting from the top (closest to output):

$$\theta_l^{(t)} = \begin{cases} \theta_l^{(t-1)} - \eta_l \nabla_{\theta_l} \mathcal{L} & \text{if } l \geq L - k(t) \\ \theta_l^{(t-1)} & \text{otherwise} \end{cases}$$

where $k(t)$ increases with training epoch $t$.

## Loss Functions and Regularization

### Cross-Entropy Loss

For classification tasks, the cross-entropy loss is:

$$\mathcal{L}_{\text{CE}} = -\frac{1}{N}\sum_{i=1}^{N}\sum_{c=1}^{C} y_{i,c} \log(\hat{y}_{i,c})$$

where $N$ is the batch size and $C$ is the number of classes.

### L2 Regularization (Weight Decay)

To prevent catastrophic forgetting and overfitting:

$$\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{CE}} + \lambda \|\theta - \theta^{\text{pre}}\|_2^2$$

This penalizes large deviations from the pre-trained weights $\theta^{\text{pre}}$.

## Implementation: Simulating Transfer Learning

We will simulate a transfer learning scenario using synthetic data and a simple neural network architecture. This demonstration illustrates the key concepts without requiring large datasets or GPU computation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
from scipy.optimize import minimize

np.random.seed(42)

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

### Synthetic Data Generation

We create two related classification tasks:
- **Source Task**: Classification with abundant labeled data
- **Target Task**: Related classification with limited labeled data

In [ ]:
def generate_classification_data(n_samples, n_features, n_classes, noise=0.1, transformation=None):
    """
    Generate synthetic classification data.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    n_features : int
        Dimensionality of feature space
    n_classes : int
        Number of classes
    noise : float
        Standard deviation of Gaussian noise
    transformation : ndarray or None
        Optional linear transformation to apply (for domain shift)
    """
    # Generate class centers
    centers = np.random.randn(n_classes, n_features) * 2
    
    # Generate samples
    X = []
    y = []
    samples_per_class = n_samples // n_classes
    
    for c in range(n_classes):
        X_c = centers[c] + np.random.randn(samples_per_class, n_features) * noise
        X.append(X_c)
        y.extend([c] * samples_per_class)
    
    X = np.vstack(X)
    y = np.array(y)
    
    # Apply domain transformation if provided
    if transformation is not None:
        X = X @ transformation
    
    # Shuffle data
    idx = np.random.permutation(len(y))
    return X[idx], y[idx], centers

# Parameters
n_features = 10
n_classes = 5
n_source = 1000  # Abundant source data
n_target = 100   # Limited target data

# Generate source domain data
X_source, y_source, source_centers = generate_classification_data(
    n_source, n_features, n_classes, noise=0.5
)

# Generate target domain data with slight domain shift
# Apply a rotation/scaling transformation to simulate domain shift
domain_shift = np.eye(n_features) + np.random.randn(n_features, n_features) * 0.1
X_target, y_target, target_centers = generate_classification_data(
    n_target, n_features, n_classes, noise=0.5, transformation=domain_shift
)

print(f"Source domain: {X_source.shape[0]} samples")
print(f"Target domain: {X_target.shape[0]} samples")
print(f"Feature dimensionality: {n_features}")
print(f"Number of classes: {n_classes}")

### Neural Network Implementation

We implement a simple two-layer neural network with ReLU activation, decomposed into feature extractor and classifier components.

In [ ]:
class SimpleNeuralNetwork:
    """
    A simple neural network for classification with transfer learning support.
    
    Architecture:
    - Feature extractor: Input -> Hidden1 -> ReLU -> Hidden2 -> ReLU
    - Classifier: Hidden2 -> Output (softmax)
    """
    
    def __init__(self, input_dim, hidden_dims, output_dim):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        
        # Initialize weights using Xavier initialization
        self.weights = {}
        
        # Feature extractor layers
        dims = [input_dim] + hidden_dims
        for i in range(len(dims) - 1):
            scale = np.sqrt(2.0 / dims[i])
            self.weights[f'W{i}'] = np.random.randn(dims[i], dims[i+1]) * scale
            self.weights[f'b{i}'] = np.zeros(dims[i+1])
        
        # Classifier layer
        scale = np.sqrt(2.0 / hidden_dims[-1])
        self.weights['W_out'] = np.random.randn(hidden_dims[-1], output_dim) * scale
        self.weights['b_out'] = np.zeros(output_dim)
        
        self.n_feature_layers = len(hidden_dims)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    def extract_features(self, X):
        """Forward pass through feature extractor only."""
        activations = [X]
        h = X
        
        for i in range(self.n_feature_layers):
            z = h @ self.weights[f'W{i}'] + self.weights[f'b{i}']
            h = self.relu(z)
            activations.append(h)
        
        return h, activations
    
    def forward(self, X):
        """Full forward pass."""
        features, activations = self.extract_features(X)
        logits = features @ self.weights['W_out'] + self.weights['b_out']
        probs = softmax(logits, axis=1)
        return probs, features, activations
    
    def compute_loss(self, X, y, reg_lambda=0.001):
        """Compute cross-entropy loss with L2 regularization."""
        probs, _, _ = self.forward(X)
        n_samples = len(y)
        
        # Cross-entropy loss
        log_probs = -np.log(probs[np.arange(n_samples), y] + 1e-10)
        ce_loss = np.mean(log_probs)
        
        # L2 regularization
        l2_loss = 0
        for key in self.weights:
            if key.startswith('W'):
                l2_loss += np.sum(self.weights[key] ** 2)
        l2_loss *= reg_lambda / 2
        
        return ce_loss + l2_loss
    
    def compute_gradients(self, X, y, reg_lambda=0.001, frozen_layers=None):
        """Compute gradients via backpropagation."""
        if frozen_layers is None:
            frozen_layers = []
        
        n_samples = len(y)
        probs, features, activations = self.forward(X)
        
        gradients = {}
        
        # Gradient of softmax + cross-entropy
        dlogits = probs.copy()
        dlogits[np.arange(n_samples), y] -= 1
        dlogits /= n_samples
        
        # Classifier layer gradients
        gradients['W_out'] = features.T @ dlogits + reg_lambda * self.weights['W_out']
        gradients['b_out'] = np.sum(dlogits, axis=0)
        
        # Backpropagate through feature extractor
        dh = dlogits @ self.weights['W_out'].T
        
        for i in range(self.n_feature_layers - 1, -1, -1):
            # ReLU derivative
            dz = dh * self.relu_derivative(activations[i+1])
            
            if i not in frozen_layers:
                gradients[f'W{i}'] = activations[i].T @ dz + reg_lambda * self.weights[f'W{i}']
                gradients[f'b{i}'] = np.sum(dz, axis=0)
            else:
                gradients[f'W{i}'] = np.zeros_like(self.weights[f'W{i}'])
                gradients[f'b{i}'] = np.zeros_like(self.weights[f'b{i}'])
            
            if i > 0:
                dh = dz @ self.weights[f'W{i}'].T
        
        return gradients
    
    def predict(self, X):
        """Predict class labels."""
        probs, _, _ = self.forward(X)
        return np.argmax(probs, axis=1)
    
    def accuracy(self, X, y):
        """Compute classification accuracy."""
        predictions = self.predict(X)
        return np.mean(predictions == y)
    
    def copy_weights(self):
        """Return a deep copy of the weights."""
        return {k: v.copy() for k, v in self.weights.items()}
    
    def load_weights(self, weights):
        """Load weights from a dictionary."""
        for k, v in weights.items():
            self.weights[k] = v.copy()

### Training Functions

In [ ]:
def train_model(model, X, y, n_epochs=100, learning_rate=0.01, batch_size=32, 
                reg_lambda=0.001, frozen_layers=None, layer_lrs=None, verbose=True):
    """
    Train the neural network using mini-batch gradient descent.
    
    Parameters:
    -----------
    model : SimpleNeuralNetwork
        The neural network to train
    X : ndarray
        Training features
    y : ndarray
        Training labels
    n_epochs : int
        Number of training epochs
    learning_rate : float
        Base learning rate
    batch_size : int
        Mini-batch size
    reg_lambda : float
        L2 regularization coefficient
    frozen_layers : list or None
        List of layer indices to freeze
    layer_lrs : dict or None
        Dictionary mapping layer names to learning rate multipliers
    
    Returns:
    --------
    history : dict
        Training history with losses and accuracies
    """
    if frozen_layers is None:
        frozen_layers = []
    if layer_lrs is None:
        layer_lrs = {}
    
    n_samples = len(y)
    n_batches = max(1, n_samples // batch_size)
    
    history = {'loss': [], 'accuracy': []}
    
    for epoch in range(n_epochs):
        # Shuffle data
        idx = np.random.permutation(n_samples)
        X_shuffled = X[idx]
        y_shuffled = y[idx]
        
        epoch_loss = 0
        
        for batch in range(n_batches):
            start = batch * batch_size
            end = min(start + batch_size, n_samples)
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # Compute gradients
            gradients = model.compute_gradients(X_batch, y_batch, reg_lambda, frozen_layers)
            
            # Update weights
            for key in model.weights:
                lr = learning_rate * layer_lrs.get(key, 1.0)
                model.weights[key] -= lr * gradients[key]
            
            epoch_loss += model.compute_loss(X_batch, y_batch, reg_lambda)
        
        epoch_loss /= n_batches
        accuracy = model.accuracy(X, y)
        
        history['loss'].append(epoch_loss)
        history['accuracy'].append(accuracy)
        
        if verbose and (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{n_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.4f}")
    
    return history

### Experiment: Comparing Transfer Learning Strategies

We will compare four approaches:
1. **From Scratch**: Training on target data only without transfer
2. **Feature Extraction**: Frozen backbone, only train classifier
3. **Full Fine-Tuning**: Update all layers with same learning rate
4. **Discriminative Fine-Tuning**: Layer-wise learning rate decay

In [ ]:
# Network architecture
hidden_dims = [32, 16]

# Step 1: Pre-train on source domain
print("="*60)
print("Pre-training on Source Domain")
print("="*60)

source_model = SimpleNeuralNetwork(n_features, hidden_dims, n_classes)
source_history = train_model(
    source_model, X_source, y_source, 
    n_epochs=100, learning_rate=0.1, batch_size=64
)

print(f"\nSource domain final accuracy: {source_model.accuracy(X_source, y_source):.4f}")
print(f"Source model on target domain (no adaptation): {source_model.accuracy(X_target, y_target):.4f}")

# Save pre-trained weights
pretrained_weights = source_model.copy_weights()

In [ ]:
# Step 2: Compare transfer learning strategies on target domain
results = {}

# Strategy 1: Training from scratch (no transfer)
print("\n" + "="*60)
print("Strategy 1: Training from Scratch (No Transfer)")
print("="*60)

scratch_model = SimpleNeuralNetwork(n_features, hidden_dims, n_classes)
scratch_history = train_model(
    scratch_model, X_target, y_target,
    n_epochs=100, learning_rate=0.1, batch_size=16
)
results['From Scratch'] = {
    'history': scratch_history,
    'final_accuracy': scratch_model.accuracy(X_target, y_target)
}
print(f"Final accuracy: {results['From Scratch']['final_accuracy']:.4f}")

In [ ]:
# Strategy 2: Feature Extraction (frozen backbone)
print("\n" + "="*60)
print("Strategy 2: Feature Extraction (Frozen Backbone)")
print("="*60)

feature_model = SimpleNeuralNetwork(n_features, hidden_dims, n_classes)
feature_model.load_weights(pretrained_weights)

# Reinitialize classifier layer
scale = np.sqrt(2.0 / hidden_dims[-1])
feature_model.weights['W_out'] = np.random.randn(hidden_dims[-1], n_classes) * scale
feature_model.weights['b_out'] = np.zeros(n_classes)

# Freeze all feature extractor layers
frozen_layers = list(range(len(hidden_dims)))

feature_history = train_model(
    feature_model, X_target, y_target,
    n_epochs=100, learning_rate=0.1, batch_size=16,
    frozen_layers=frozen_layers
)
results['Feature Extraction'] = {
    'history': feature_history,
    'final_accuracy': feature_model.accuracy(X_target, y_target)
}
print(f"Final accuracy: {results['Feature Extraction']['final_accuracy']:.4f}")

In [ ]:
# Strategy 3: Full Fine-Tuning
print("\n" + "="*60)
print("Strategy 3: Full Fine-Tuning")
print("="*60)

full_ft_model = SimpleNeuralNetwork(n_features, hidden_dims, n_classes)
full_ft_model.load_weights(pretrained_weights)

# Reinitialize classifier layer
scale = np.sqrt(2.0 / hidden_dims[-1])
full_ft_model.weights['W_out'] = np.random.randn(hidden_dims[-1], n_classes) * scale
full_ft_model.weights['b_out'] = np.zeros(n_classes)

full_ft_history = train_model(
    full_ft_model, X_target, y_target,
    n_epochs=100, learning_rate=0.01, batch_size=16  # Lower learning rate
)
results['Full Fine-Tuning'] = {
    'history': full_ft_history,
    'final_accuracy': full_ft_model.accuracy(X_target, y_target)
}
print(f"Final accuracy: {results['Full Fine-Tuning']['final_accuracy']:.4f}")

In [ ]:
# Strategy 4: Discriminative Fine-Tuning (layer-wise learning rates)
print("\n" + "="*60)
print("Strategy 4: Discriminative Fine-Tuning (Layer-wise LR)")
print("="*60)

disc_model = SimpleNeuralNetwork(n_features, hidden_dims, n_classes)
disc_model.load_weights(pretrained_weights)

# Reinitialize classifier layer
scale = np.sqrt(2.0 / hidden_dims[-1])
disc_model.weights['W_out'] = np.random.randn(hidden_dims[-1], n_classes) * scale
disc_model.weights['b_out'] = np.zeros(n_classes)

# Layer-wise learning rate decay (gamma = 0.5)
gamma = 0.5
layer_lrs = {
    'W0': gamma**2, 'b0': gamma**2,  # First layer (smallest LR)
    'W1': gamma**1, 'b1': gamma**1,  # Second layer
    'W_out': 1.0, 'b_out': 1.0       # Classifier (full LR)
}

disc_history = train_model(
    disc_model, X_target, y_target,
    n_epochs=100, learning_rate=0.05, batch_size=16,
    layer_lrs=layer_lrs
)
results['Discriminative FT'] = {
    'history': disc_history,
    'final_accuracy': disc_model.accuracy(X_target, y_target)
}
print(f"Final accuracy: {results['Discriminative FT']['final_accuracy']:.4f}")

### Results Visualization

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = {
    'From Scratch': '#e74c3c',
    'Feature Extraction': '#3498db',
    'Full Fine-Tuning': '#2ecc71',
    'Discriminative FT': '#9b59b6'
}

# Plot 1: Training Loss Curves
ax1 = axes[0, 0]
for name, data in results.items():
    ax1.plot(data['history']['loss'], label=name, color=colors[name], linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Comparison')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 100)

# Plot 2: Training Accuracy Curves
ax2 = axes[0, 1]
for name, data in results.items():
    ax2.plot(data['history']['accuracy'], label=name, color=colors[name], linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy Comparison')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 100)
ax2.set_ylim(0, 1.05)

# Plot 3: Final Accuracy Bar Chart
ax3 = axes[1, 0]
names = list(results.keys())
accuracies = [results[name]['final_accuracy'] for name in names]
bar_colors = [colors[name] for name in names]
bars = ax3.bar(names, accuracies, color=bar_colors, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Final Accuracy')
ax3.set_title('Final Accuracy by Strategy')
ax3.set_ylim(0, 1.0)
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{acc:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Rotate x-axis labels for better readability
ax3.set_xticklabels(names, rotation=15, ha='right')

# Plot 4: Convergence Speed (epochs to reach 80% of final accuracy)
ax4 = axes[1, 1]

def epochs_to_threshold(history, threshold_fraction=0.9):
    """Find number of epochs to reach threshold fraction of final accuracy."""
    final_acc = history['accuracy'][-1]
    threshold = threshold_fraction * final_acc
    for i, acc in enumerate(history['accuracy']):
        if acc >= threshold:
            return i + 1
    return len(history['accuracy'])

convergence_epochs = [epochs_to_threshold(results[name]['history']) for name in names]
bars = ax4.bar(names, convergence_epochs, color=bar_colors, edgecolor='black', linewidth=1.5)
ax4.set_ylabel('Epochs to 90% of Final Accuracy')
ax4.set_title('Convergence Speed Comparison')
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, epochs in zip(bars, convergence_epochs):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{epochs}', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax4.set_xticklabels(names, rotation=15, ha='right')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to 'plot.png'")

## Summary and Key Findings

### Results Analysis

In [ ]:
print("\n" + "="*70)
print("TRANSFER LEARNING EXPERIMENT SUMMARY")
print("="*70)
print(f"\nSource domain samples: {n_source}")
print(f"Target domain samples: {n_target} (10x less data)")
print(f"\n{'Strategy':<25} {'Final Accuracy':<18} {'Convergence (epochs)'}")
print("-"*60)

for name in names:
    acc = results[name]['final_accuracy']
    conv = epochs_to_threshold(results[name]['history'])
    print(f"{name:<25} {acc:<18.4f} {conv}")

print("\n" + "="*70)
print("KEY OBSERVATIONS")
print("="*70)
print("""
1. **Transfer Learning Benefit**: All transfer learning strategies outperform
   training from scratch when target data is limited.

2. **Feature Extraction**: Provides stable, fast convergence by leveraging
   learned representations. Best when source and target domains are similar.

3. **Full Fine-Tuning**: Can achieve highest accuracy but requires careful
   learning rate selection to avoid catastrophic forgetting.

4. **Discriminative Fine-Tuning**: Balances adaptation and preservation by
   using smaller learning rates for earlier layers that capture general features.

5. **Convergence Speed**: Transfer learning methods typically converge faster
   than training from scratch due to better initialization.
""")

## Mathematical Summary

The key insight of transfer learning is that neural networks learn hierarchical representations:

- **Early layers** learn general, task-agnostic features (edges, textures, basic patterns)
- **Later layers** learn task-specific features

The optimal fine-tuning strategy depends on:

1. **Domain similarity**: $\text{sim}(\mathcal{D}_S, \mathcal{D}_T)$
   - High similarity → Feature extraction or conservative fine-tuning
   - Low similarity → Full fine-tuning with domain adaptation

2. **Target data size**: $|\mathcal{D}_T|$
   - Small → Freeze more layers to prevent overfitting
   - Large → Fine-tune more layers for better adaptation

The discriminative fine-tuning approach provides a principled middle ground by respecting the hierarchical nature of learned representations:

$$\eta_l = \eta_{\text{base}} \cdot \gamma^{L-l}, \quad \gamma \in (0, 1)$$

This allows task-specific layers to adapt quickly while preserving the general-purpose features that transfer well across domains.